In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# import warnings
import warnings
# filter warnings
warnings.filterwarnings('ignore')




# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
print(os.listdir("../input"))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
train=pd.read_csv('../input/digit-recognizer/train.csv')
test=pd.read_csv('../input/digit-recognizer/test.csv')

In [1]:
train

28*28 piksel 42000 adet fotograf

Labellerımızı ayrı bır in64dızı ıcıne aldık

In [1]:
label_train=train["label"]

In [1]:
label_train

Labellerı ayırmıstık burda sadece fotoları bırakıyoruz

In [1]:
image_train=train.drop(labels=["label"], axis=1)

In [1]:
image_train

Datamızda filtelencek veri yok cunku her sey tamamıyla olmus vazıyette kontrol edilebilir //// Sımdı Normal/Reshape/Label Encodıng yapalım

In [1]:
image_train=image_train/255.0


In [1]:
image_train

In [1]:
label_train

In [1]:
label_train.value_counts()

Normalıze kısmı tamamlandı şimdi rehsahpe yapalım degerleri.

In [1]:
image_train=image_train.values.reshape(-1,28,28,1)
test=test.values.reshape(-1,28,28,1)

In [1]:
print("image shape",image_train.shape)
print("test shape",test.shape)

Kerasın ıstedıgı gıbı bır versıyona cevırdık///sımdı label encodıng kısmındayız

In [1]:
# Label Encoding 
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding#kategorık sınıflandırmalar ıcın gereklı olan kod blogu
en_label_train = to_categorical(label_train, num_classes = 10)

Normalızatıon/Reshape/Encodıng yapıldı artık datamız egıtım ıcın hazır

Train ve Test Split yapalım

In [1]:
from sklearn.model_selection import train_test_split
image_train, image_val,en_label_train,en_label_val = train_test_split(image_train, en_label_train, test_size = 0.1, random_state=2)
print("image_train shape",image_train.shape)
print("image_test shape",image_val.shape)
print("label_train shape",en_label_train.shape)
print("label_test shape",en_label_val.shape)

In [1]:
plt.imshow(image_train[3][:,:,0],cmap='gray')
plt.show()

Algorıtmalar kısmı

In [1]:
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

model = Sequential()
#
model.add(Conv2D(filters = 8, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
#
model.add(Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))
#fully connected
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

*3 aşama
***model.compile
***datageneretor.fit
**model.fit_generator

In [1]:
#ogrenme rate//bu sekılde ogrenme oranı degısıyor
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

In [1]:
#modelımızı kompole edelım categorık ayrısım yapıcaz ve dogruluga bakıcaz optımızerı belırledık zaten
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [1]:
epochs = 10  # epoch 10 olsun batch_size 250 bunlar degısebılır degerler belırledık
batch_size = 250

In [1]:
#degısebılır degerler datageneretor image_train icinden alıyoruz
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # dimesion reduction
        rotation_range=5,  # randomly rotate images in the range 5 degrees
        zoom_range = 0.1, # Randomly zoom image 10%
        width_shift_range=0.1,  # randomly shift images horizontally 10%
        height_shift_range=0.1,  # randomly shift images vertically 10%
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(image_train)

modelımızı ayarlayalım degeerlerımızı gırmıstık zaten model fit

In [1]:
history = model.fit_generator(datagen.flow(image_train,en_label_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (image_val,en_label_val), steps_per_epoch=image_train.shape[0] // batch_size)#151 degeri dondurur

In [1]:
plt.plot(history.history['val_loss'], color='r', label="validation loss")
plt.title("Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [1]:

import seaborn as sns
Y_pred = model.predict(image_val)
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
Y_true = np.argmax(en_label_val,axis = 1) 
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()